In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml


np.random.seed(0)
%matplotlib inline

In [2]:
#Load the training data
X = np.genfromtxt("data/X_train.txt", delimiter=None) #load the data
Y= np.genfromtxt("data/Y_train.txt", delimiter=None) #load the data

#and the test features
Xte= np.genfromtxt("data/X_test.txt", delimiter=None) #load the data

Xtr,Xva,Ytr,Yva = ml.splitData(X,Y); # Default is 80% training/20% validation
Xtr, Ytr = ml.shuffleData(Xtr, Ytr)

#Taking a subset for testing (get rid of this later)
#Xtr, Ytr = Xtr[:4000], Ytr[:4000]


In [8]:
#polynomial degree
degree = 0.5

### Rescale Training data (Xtr)
# Create polynomial features up to "degree"
XtrP = ml.transforms.fpoly(Xtr, degree, bias=False)

# Rescale the data matrix so that the features have similar ranges / variance
XtrP,params = ml.transforms.rescale(XtrP) # "params" returns the transformation parameters (shift & scale)

###Rescale test data (Xte)
#Apply the same polynomial expansion & scaling transformation as XtrP
XteP, _ = ml.transforms.rescale(ml.transforms.fpoly(Xte,degree, False),params)

### Rescale Validation Data (Xva)
#Apply the same polynomial expansion & scaling transformation as XtrP
XvaP, _ =  ml.transforms.rescale(ml.transforms.fpoly(Xva,degree, False),params)

#check shapes
print(XtrP.shape, XvaP.shape, XteP.shape)

TypeError: 'float' object cannot be interpreted as an integer

In [7]:
# Train the model on the scaled feature matrix:
learner = ml.linearC.linearClassify()
learner.train(XtrP, Ytr, reg=0.1, initStep=0.01, stopTol=1e-6, stopIter=100)

probs = learner.predictSoft(XteP)
print(probs[:5])

#Use XtrP and XvaP
print("{0:>15}: {1:.4f}".format('Train AUC',learner.auc(XtrP, Ytr)))
print("{0:>15}: {1:.4f}".format('Validation AUC', learner.auc(XvaP, Yva)))

/Users/deejayslay/Documents/CS 178/final project/mltools/base.py:96: RuntimeWarning: divide by zero encountered in log
  return - np.mean( np.log( P[ np.arange(M), Y ] ) ) # evaluate
/Users/deejayslay/Documents/CS 178/final project/mltools/linearC.py:134: RuntimeWarning: invalid value encountered in double_scalars
  done = (it > stopIter) or ( (it>1) and (abs(Jsur[-1]-Jsur[-2])<stopTol) )


[[0.67867453 0.32132547]
 [0.56008662 0.43991338]
 [0.73189031 0.26810969]
 [0.80349101 0.19650899]
 [0.76024431 0.23975569]]
      Train AUC: 0.6602
 Validation AUC: 0.6569


## Preparing the submission text

In [5]:
# Create the data for submission by taking the P(Y=1) column from probs and just add a running index as the first column.
Y_sub = np.vstack([np.arange(XteP.shape[0]), probs[:, 1]]).T

# We specify the header (ID, Prob1) and also specify the comments as '' so the header won't be commented out with
# the # sign.
np.savetxt('data/Y_sub.txt', Y_sub, '%d, %.5f', header='ID,Prob1', comments='', delimiter=',')